In [62]:
import pandas as pd

In [79]:
#data = pd.read_csv('convert_MCMF_ALL_TIME_DATA.csv', sep = '\t')
data = pd.read_csv('mcmf_cleaned.csv')

In [128]:
df = data[["Org Name", "Description", "Program Url", "Start Date", "End Date", "Min Age", "Max Age", "Category Name", "Meeting Type", "Geographic Cluster Name", "Program Pays Participants", "Program Has Scholarships", "Program Provides Transportation", "Program Provides Free Food", "Program Price"]]

df['Program Price'] = df['Program Price'].map({0 : 'Free', 1 : '$50 or Less', 2 : 'More Than $50', 3 : 'Unknown'})
df['Program Provides Free Food'] = df['Program Provides Free Food'].map({False : 'NO', True : 'YES', np.nan : np.nan})

startMonth = []
year = []
for item in df["Start Date"]:
    result = item.split("/", 1)[0]
    startMonth.append(result)
    y = item.rsplit("/", 1)[-1]
    year.append(y)
df["Start Month"] = startMonth
df["Start Year"] = year

endMonth = []
endYear = []
for entry in df["End Date"]:
    m = entry.split("/", 1)[0]
    endMonth.append(m)
    y2 = entry.rsplit("/", 1)[-1]
    endYear.append(y2)
df["End Month"] = endMonth
df["End Year"] = endYear

#new = []
#for i in range(0,len(df.iloc[:, 0])):
#    new.append((int(df.loc[i,"Start Month"]) - int(df.loc[i,"End Month"])) > 0)
    
#df["Overlap"] = new

df['Start Month'] = df['Start Month'].astype('int')
df['Start Year'] = df['Start Year'].astype('int')
df['End Month'] = df['End Month'].astype('int')
df['End Year'] = df['End Year'].astype('int')

use = df[["Org Name", "Description", "Program Url", "Start Month", "End Month", "Start Year", "End Year", "Min Age", "Max Age", "Category Name", "Meeting Type", "Geographic Cluster Name", "Program Pays Participants", "Program Has Scholarships", "Program Provides Transportation", "Program Provides Free Food", "Program Price"]]

#test = df[["Org Name", "Description", "Program Url"]]

In [127]:
u = use['Start Year'].value_counts()
print(u)

22    34180
21    22605
23    15918
20    11882
Name: Start Year, dtype: int64


In [129]:
import dash
import dash_core_components as dcc
from dash import html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import math
import numpy as np

# Load the data
df = use

# Get unique values from the remaining columns
categories = df['Category Name'].unique()
categories = sorted(categories, key=lambda x: (isinstance(x, str), str(x)))
meeting_types = df['Meeting Type'].unique()
clusters = df['Geographic Cluster Name'].unique()
clusters = sorted(clusters, key=lambda x: (isinstance(x, str), str(x)))
pays_participants = df['Program Pays Participants'].unique()
has_scholarships = df['Program Has Scholarships'].unique()
provides_transportation = df['Program Provides Transportation'].unique()
provides_free_food = df['Program Provides Free Food'].unique()
program_price = df['Program Price'].unique()

#free_food = [{False:'False'}, {True:'True'}, {np.nan:np.nan}]

"""category_options = [{'label': category, 'value': category} for category in categories]
sorted_cats = sorted(category_options, key=lambda d: list(d.keys())[0])

cluster_options = [{'label': cluster, 'value': cluster} for cluster in clusters]
sorted_clust = sorted(cluster_options, key=lambda d: list(d.keys())[0])"""

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H1('MCMF Data Filtering App'),
    dcc.Dropdown(
        id='category-dropdown',
        options=[{'label': category, 'value': category} for category in categories],
        placeholder='Select a program category'
    ),
    html.Br(),
    dcc.Input(
        id='age-input',
        type='number',
        placeholder='Enter an age'
    ),
    html.Br(),
    dcc.Dropdown(
        id='meeting-type-dropdown',
        options=[{'label': meeting_type, 'value': meeting_type} for meeting_type in meeting_types],
        placeholder='Select a meeting modality'
    ),
    html.Br(),
    dcc.Dropdown(
        id='cluster-dropdown',
        options=[{'label': cluster, 'value': cluster} for cluster in clusters],
        placeholder='Select a neighborhood'
    ),
    html.Br(),
    dcc.Dropdown(
        id='pays-participants-dropdown',
        options=[{'label': option, 'value': option} for option in pays_participants],
        placeholder='Pays participants?'
    ),
    html.Br(),
    dcc.Dropdown(
        id='has-scholarships-dropdown',
        options=[{'label': option, 'value': option} for option in has_scholarships],
        placeholder='Has scholarships?'
    ),
    html.Br(),
    dcc.Dropdown(
        id='provides-transportation-dropdown',
        options=[{'label': option, 'value': option} for option in provides_transportation],
        placeholder='Provides transportation?'
    ),
    html.Br(),
    dcc.Dropdown(
        id='provides-free-food-dropdown',
        options=[{'label': option, 'value': option} for option in provides_free_food],
        placeholder='Provides free food?'
    ),
    html.Br(),
    dcc.Dropdown(
        id='program-price-dropdown',
        options=[{'label': option, 'value': option} for option in program_price],
        placeholder='Select a program price'
    ),
    html.Br(),
    dcc.Input(
        id='month-input',
        type='number',
        min=1,
        max=12,
        placeholder='Enter a month (1-12)'
    ),
    html.Br(),
    html.Button('Enter', id='enter-button', n_clicks=0),
    html.Br(),
    html.Div(id='row-count'),
    html.Div([
        dcc.Graph(id='line-graph')
    ]),
    html.Div(id='table-container')
])

# Define the callback function
@app.callback(
    [Output('row-count', 'children'), Output('line-graph', 'figure'), Output('table-container', 'children')],
    [Input('enter-button', 'n_clicks')],
    [State('category-dropdown', 'value'),
     State('age-input', 'value'),
     State('meeting-type-dropdown', 'value'),
     State('cluster-dropdown', 'value'),
     State('pays-participants-dropdown', 'value'),
     State('has-scholarships-dropdown', 'value'),
     State('provides-transportation-dropdown', 'value'),
     State('provides-free-food-dropdown', 'value'),
     State('program-price-dropdown', 'value'),
     State('month-input', 'value')]
)
def update_table(n_clicks, category, age, meeting_type, cluster, pays_participants, has_scholarships,
                 provides_transportation, provides_free_food, program_price, month):
    if n_clicks > 0:  # Table is displayed only when the button is clicked
        filtered_data = df.copy()  # Start with a copy of the entire dataset
        
        # Apply filters if values are provided
        if category:
            filtered_data = filtered_data[filtered_data['Category Name'] == category]
        if age:
            filtered_data = filtered_data[(filtered_data['Min Age'] <= age) & (filtered_data['Max Age'] >= age)]
        if meeting_type:
            filtered_data = filtered_data[filtered_data['Meeting Type'] == meeting_type]
        if cluster:
            filtered_data = filtered_data[filtered_data['Geographic Cluster Name'] == cluster]
        if pays_participants:
            filtered_data = filtered_data[filtered_data['Program Pays Participants'] == pays_participants]
        if has_scholarships:
            filtered_data = filtered_data[filtered_data['Program Has Scholarships'] == has_scholarships]
        if provides_transportation:
            filtered_data = filtered_data[filtered_data['Program Provides Transportation'] == provides_transportation]
        if provides_free_food:
            filtered_data = filtered_data[filtered_data['Program Provides Free Food'] == provides_free_food]
        if program_price:
            filtered_data = filtered_data[filtered_data['Program Price'] == program_price]
        if month:
            filtered_data = filtered_data[((filtered_data['Start Month'] <= month) & (filtered_data['End Month'] >= month)) | ((filtered_data['Start Month'] > filtered_data['End Month']) & (filtered_data['End Month'] >= month)) | ((filtered_data['Start Month'] > filtered_data['End Month']) & (filtered_data['Start Month'] <= month))]
            """start_month = filtered_data['Start Month']
            end_month = filtered_data['End Month']
            if start_month <= end_month:
                filtered_data = filtered_data[(filtered_data['Start Month'] <= month) &
                                              (filtered_data['End Month'] >= month)]
            else:
                filtered_data = filtered_data[(filtered_data['Start Month'] <= month) |
                                              (filtered_data['End Month'] >= month)]"""
        
        sy_20 = len(filtered_data[filtered_data['Start Year'] == 20])
        sy_21 = len(filtered_data[filtered_data['Start Year'] == 21])
        sy_22 = len(filtered_data[filtered_data['Start Year'] == 22])
        sy_23 = len(filtered_data[filtered_data['Start Year'] == 23])
        pred = int((sy_22*.7 + sy_20*.1 + sy_21*.1 + sy_23*.1))
        
        table = generate_table(filtered_data)
        
        # Create line graph
        line_graph = go.Figure(data=go.Scatter(x=["2020", "2021", "2022", "2023", "Future"], y=[sy_20,sy_21,sy_22,sy_23,pred], mode='lines'))

        # Update the layout of the line graph
        line_graph.update_layout(
            title="Program Matches by Year",
            xaxis_title="Year",
            yaxis_title="Number of Matching Programs"
        )
        
        """c20, c21, c22, c23 = 0
        for yr in filtered_data["Start Year"]:
            if yr == 20:
                c20+=1
            elif yr == 21:
                c21+=1
            elif yr == 22:
                c22+=1
            elif yr == 23:
                c23+=1"""
        
        row_count = html.Div([
            html.Div(f"Number of total program matches: {len(filtered_data)}."),
            html.Div(f"Number of total 2020 matches: {sy_20}."),
            html.Div(f"Number of total 2021 matches: {sy_21}."),
            html.Div(f"Number of total 2022 matches: {sy_22}."),
            html.Div(f"Number of total 2023 matches: {sy_23}."),
            html.Div(f""),
            html.Div(f"In future years, we predict the number of matching programs to be: {pred}.")
        ])
        return row_count, line_graph, table
    else:
        return '', ''  # Empty string when the button is not clicked

    
# Helper function to generate HTML table
def generate_table(dataframe, max_rows=10):
    table_header = [
        html.Thead(html.Tr([html.Th(col) for col in dataframe.columns]))
    ]
    
    table_body = []
    for i in range(min(len(dataframe), max_rows)):
        table_body.append(
            html.Tr([html.Td(dataframe.iloc[i][col]) for col in dataframe.columns])
        )
    
    table = html.Table(table_header + table_body, className='data-table')
    return table

# Run the app
app.run_server(port=8102)


Dash is running on http://127.0.0.1:8102/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8102
Press CTRL+C to quit
127.0.0.1 - - [25/May/2023 22:35:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2023 22:35:31] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_9_3m1684536146.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2023 22:35:31] "GET /_dash-component-suites/dash/deps/react@16.v2_9_3m1684536146.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2023 22:35:31] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_9_3m1684536146.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2023 22:35:31] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_9_3m1684536146.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2023 22:35:31] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_9_2m1684536145.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2023 22:35:31] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_9_3m1684536145.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2023 22:35:31] "GET /_dash-component-suit

" 304 -
127.0.0.1 - - [26/May/2023 09:25:21] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [26/May/2023 09:25:21] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 09:29:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 09:30:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 09:30:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 09:30:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 09:30:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 09:30:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 09:39:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 09:39:25] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 09:39:25] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 09:39:25] "GET /_favicon.ico?v=2.9.3 HTTP/1.1" 200 -
127.0.0.1 - 

127.0.0.1 - - [26/May/2023 11:27:01] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [26/May/2023 11:33:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 11:35:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 11:35:04] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 11:35:04] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 11:35:04] "GET /_favicon.ico?v=2.9.3 HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 11:35:04] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1[2023-05-26 11:35:04,322] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/matthewholleran-meyer/anaconda3/lib/python3.10/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/matthewholleran-meyer/anaconda3/lib/python3.10/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  F

In [114]:
print(categories)

['Academic Support' 'Work & Career ' 'Music & Art' 'Managing Money'
 'Helping Your Community' 'Reading & Writing' 'Sports & Wellness'
 'Computers' 'Science ' 'Building & Fixing Things' 'Performance'
 'Science & Math' 'Food' 'Math' 'Digital Media' 'Nature' 'Healthcare'
 'Transportation' 'Social Studies ' 'Teaching' 'Law'
 'Customer/Human Service' nan]


In [120]:
import numpy as np
hi = ["uiwf", "wuyfy", "dfbnoi", np.nan, "fuib"]
hi2 = sorted(hi, key=lambda x: (isinstance(x, str), str(x)))
print(hi2)

[nan, 'dfbnoi', 'fuib', 'uiwf', 'wuyfy']
